In [0]:
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt
#import seaborn as sns

**Importing Data**

In [3]:
# For normal inputting data if running on local System

df1= pd.read_csv('complaints.txt',sep=';')
df2= pd.read_csv('credit.txt',sep=';')
df3= pd.read_csv('customers.txt',sep=';')
df4= pd.read_csv('delivery.txt',sep=';')
df5= pd.read_csv('formula.txt',sep=';')
df6= pd.read_csv('subscriptions.txt',delim_whitespace=True)

"\ndf1= pd.read_csv('complaints.txt',sep=';')\ndf2= pd.read_csv('credit.txt',sep=';')\ndf3= pd.read_csv('customers.txt',sep=';')\ndf4= pd.read_csv('delivery.txt',sep=';')\ndf5= pd.read_csv('formula.txt',sep=';')\ndf6= pd.read_csv('subscriptions.txt',delim_whitespace=True)"

In [5]:
# For inputting data to Google Colab

from google.colab import files
import io

uploaded=files.upload()
df11 = pd.read_csv(io.StringIO(uploaded['complaints.txt'].decode('utf-8')),sep=';')

uploaded=files.upload()
df21 = pd.read_csv(io.StringIO(uploaded['credit.txt'].decode('utf-8')),sep=';')

uploaded=files.upload()
df31 = pd.read_csv(io.StringIO(uploaded['customers.txt'].decode('utf-8')),sep=';')

uploaded=files.upload()
df41 = pd.read_csv(io.StringIO(uploaded['delivery.txt'].decode('utf-8')),sep=';')

uploaded=files.upload()
df51 = pd.read_csv(io.StringIO(uploaded['formula.txt'].decode('utf-8')),sep=';')

uploaded=files.upload()
df61 = pd.read_csv(io.StringIO(uploaded['subscriptions.txt'].decode('utf-8')),delim_whitespace=True)

Saving complaints.txt to complaints (4).txt


Saving credit.txt to credit (4).txt


Saving customers.txt to customers (4).txt


Saving delivery.txt to delivery (4).txt


Saving formula.txt to formula (3).txt


Saving subscriptions.txt to subscriptions (3).txt


In [0]:
df1=pd.DataFrame.copy(df11)
df2=pd.DataFrame.copy(df21)
df3=pd.DataFrame.copy(df31)
df4=pd.DataFrame.copy(df41)
df5=pd.DataFrame.copy(df51)
df6=pd.DataFrame.copy(df61)

**Importing Subcription Data and aligning the data**

In [0]:
col=df6.columns
df6=df6.drop('TotalCredit',axis=1)
df6.reset_index(inplace=True)
df6.columns=col

for i in range(4,len(df6.columns)-1):
    df6.iloc[0,i]=df6.iloc[0,i+1]

for i in range(len(df6)):
    if (df6.iloc[i,7]=='BT'):
        for j in range(len(df6.columns)-3,6,-1):
            df6.iloc[i,j+2]=df6.iloc[i,j]
        df6.iloc[i,7]=np.nan
        df6.iloc[i,8]=np.nan
    if (df6.iloc[i,8]=='BT' or df6.iloc[i,8]=='DD'):
        for j in range(len(df6.columns)-2,7,-1):
            df6.iloc[i,j+1]=df6.iloc[i,j]
        df6.iloc[i,8]=np.nan
    if (df6.iloc[i,10]=='Not'):
        df6.iloc[i,10]='Not Paid'
        df6.iloc[i,11]=np.nan
    x=str(df6.iloc[i,7])
    if (x.rfind('/')!=-1):
        df6.iloc[i,8]=df6.iloc[i,7]
        df6.iloc[i,7]=np.nan
    x=str(df6.iloc[i,12])
    if (x.rfind('/')!=-1):
        df6.iloc[i,12]='99999999'
    if(type(df6.loc[i,'NetNewspaperPrice'])!=float):
        x=str(df6.loc[i,'NetNewspaperPrice'])
        if ((df6.loc[i,'NetNewspaperPrice'].count('.')>1)|(x.rfind('-')!=-1)):
            df6.loc[i,'NetNewspaperPrice']=np.nan
    if(type(df6.loc[i,'FormulaDiscount'])!=float):
        if (df6.loc[i,'FormulaDiscount'].count('.')>1):
            df6.loc[i,'FormulaDiscount']=np.nan
    if(type(df6.loc[i,'TotalDiscount'])!=float):
        if (df6.loc[i,'TotalDiscount'].count('.')>1):
            df6.loc[i,'TotalDiscount']=np.nan
    if(type(df6.loc[i,'ProductDiscount'])!=float):
        if (df6.loc[i,'ProductDiscount'].count('.')>1):
            df6.loc[i,'ProductDiscount']=np.nan
    if(type(df6.loc[i,'TotalCredit'])!=float):
        if (df6.loc[i,'TotalCredit'].count('.')>1):
            df6.loc[i,'TotalCredit']=np.nan


In [0]:
x=['NetNewspaperPrice','FormulaDiscount','TotalDiscount','ProductDiscount','TotalCredit']
for i in x:
    df6[i]=df6[i].astype(float)

In [0]:
df6=df6.drop(df6.index[15459:15471])

df6.FormulaID=df6.FormulaID.astype(int) 
df6.RenewalDate=np.where(df6.RenewalDate.isna()==True,'01/01/2099',df6.RenewalDate)
df6.PaymentDate=np.where(df6.PaymentDate.isna()==True,'01/01/2099',df6.PaymentDate)

x=['StartDate','EndDate','RenewalDate','PaymentDate']
for i in x:
    df6[i]=pd.to_datetime(df6[i],format='%d/%m/%Y')

df4.StartDate=pd.to_datetime(df4.StartDate,format='%d/%m/%Y')        
df4.EndDate=pd.to_datetime(df4.EndDate,format='%d/%m/%Y')    
df3.DOB=pd.to_datetime(df3.DOB,format='%d/%m/%Y')

**Merging the data and droping variables with high NaN value**

In [0]:
df=pd.merge(df6,df5,how='left',on='FormulaID')
df=pd.merge(df4,df,how='left',on='SubscriptionID')
df=pd.merge(df,df3,how='left',on='CustomerID')
df=pd.merge(df,df2,how='left',on='SubscriptionID')
df=pd.merge(df,df1,how='left',on=['CustomerID','ProductID'])

# NetFormulaPrice, TotalPrice and GrossFormulaPrice removed due to high correlation
df=df.drop(['FeedbackType','SolutionType','ComplaintDate','ComplaintID','ComplaintType','TotalPrice',
            'NbrNewspapers_y','NbrStart','Amount','CreditSource','ProcessingDate','GrossFormulaPrice',
            'ActionType','Duration','FormulaType','FormulaCode','DeliveryContext','NetFormulaPrice'],axis=1)

df.NbrNewspapers_x=df.NbrNewspapers_x.astype(int)
df['Churn']=np.where((df.RenewalDate>df.StartDate_y) & (df.RenewalDate<df.EndDate_y),0,1)

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 62344 entries, 0 to 62343
Data columns (total 27 columns):
SubscriptionID       62344 non-null int64
DeliveryType         62344 non-null object
DeliveryClass        53157 non-null object
StartDate_x          62344 non-null datetime64[ns]
EndDate_x            62344 non-null datetime64[ns]
CustomerID           62344 non-null int64
ProductID            62344 non-null float64
Pattern              62344 non-null float64
StartDate_y          62344 non-null datetime64[ns]
EndDate_y            62344 non-null datetime64[ns]
NbrNewspapers_x      62344 non-null int64
RenewalDate          62344 non-null datetime64[ns]
PaymentType          62344 non-null object
PaymentStatus        62344 non-null object
PaymentDate          62344 non-null datetime64[ns]
FormulaID            62344 non-null int64
NetNewspaperPrice    56330 non-null float64
ProductDiscount      62301 non-null float64
FormulaDiscount      59440 non-null float64
TotalDiscount        5939

**Profiling Data**

In [0]:
desc=df.describe().T
desc['1%']=df.quantile(.01)
desc['5%']=df.quantile(.05)
desc['95%']=df.quantile(.95)
desc['99%']=df.quantile(.99)
desc['Q3-Q1']=desc['75%']-desc['25%']
desc['UB']=desc['75%']+(desc['Q3-Q1']*1.5)
desc['LB']=desc['25%']-(desc['Q3-Q1']*1.5)
desc=desc.loc[:,['count','mean','std','min','max','1%','5%','25%','50%','75%','95%','99%','Q3-Q1','LB','UB']]
#desc.to_csv('desc.csv')

**Handling Missing Values**

In [0]:
#Delivery Class
col=pd.get_dummies(df['DeliveryClass'],prefix='DeliveryClass')
df=pd.concat([df,col],axis=1)
df=df.drop('DeliveryClass',axis=1)

#NetNewspaperPrice
df.NetNewspaperPrice=df.NetNewspaperPrice.fillna(df.NetNewspaperPrice.median())

#TotalDiscount
df.TotalDiscount=df.TotalDiscount.fillna(df.TotalDiscount.mode())

#FormulaDiscount
df.FormulaDiscount=df.FormulaDiscount.fillna(df.FormulaDiscount.mode())

#Gender
df=df.dropna()
df.reset_index(drop=True,inplace=True)

**Removing Outliers**

In [0]:
#NbrNewspapers_x
df['NbrNewspapers_x']=np.clip(df['NbrNewspapers_x'],desc.loc['NbrNewspapers_x','1%'],desc.loc['NbrNewspapers_x','99%'])

#NetNewspaperPrice
df['NetNewspaperPrice']=np.clip(df['NetNewspaperPrice'],desc.loc['NetNewspaperPrice','1%'],desc.loc['NetNewspaperPrice','99%'])

#TotalCredit
df['TotalCredit']=df['TotalCredit'].clip_lower(desc.loc['TotalCredit','1%'])

**Preprocessing-Label Encoding and dummy variable**

In [0]:
from sklearn.preprocessing import LabelEncoder

lc=LabelEncoder()
df['Gender']=lc.fit_transform(df['Gender'])
df['PaymentStatus']=lc.fit_transform(df['PaymentStatus'])
df['PaymentType']=lc.fit_transform(df['PaymentType'])

In [0]:
#DeliveryType
col=pd.get_dummies(df['DeliveryType'],prefix='DeliveryType')
col=col.drop(col.columns[0],axis=1)
df=pd.concat([df,col],axis=1)
df=df.drop('DeliveryType',axis=1)

#District
col=pd.get_dummies(df['District'],prefix='District')
col=col.drop(col.columns[0],axis=1)
df=pd.concat([df,col],axis=1)
df=df.drop('District',axis=1)

In [0]:
df=df.drop_duplicates()
df=df.reset_index(drop=True)

In [0]:
#Converting Timestamp to ordinal

for i in range(len(df)):
    df.loc[i,'DOB']=df.loc[i,'DOB'].toordinal()
    df.loc[i,'RenewalDate']=df.loc[i,'RenewalDate'].toordinal()
    df.loc[i,'PaymentDate']=df.loc[i,'PaymentDate'].toordinal()
    df.loc[i,'StartDate_x']=df.loc[i,'StartDate_x'].toordinal()
    df.loc[i,'StartDate_y']=df.loc[i,'StartDate_y'].toordinal()
    df.loc[i,'EndDate_x']=df.loc[i,'EndDate_x'].toordinal()
    df.loc[i,'EndDate_y']=df.loc[i,'EndDate_y'].toordinal()    

In [0]:
#Splitting Output Variable

y=df['Churn']
df.drop('Churn',axis=1,inplace=True)

**Splitting data into Train and test**

In [0]:
from sklearn.model_selection import train_test_split

x_train,x_test,y_train,y_test=train_test_split(df,y,test_size=0.2,random_state=0)

**Model 1: Logistic Regression**

In [42]:
from sklearn.linear_model import LogisticRegression

model=LogisticRegression(random_state=0,C=10,penalty='l1'n_jobs=-1)
model.fit(x_train,y_train)

LogisticRegression(C=10, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=0, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [0]:
# Predicting Output

ypred=model.predict(x_test)

In [0]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score,confusion_matrix,f1_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import average_precision_score


cv_acc=cross_val_score(estimator=model,X=x_train,y=y_train,cv=10)
cm=confusion_matrix(y_test,ypred)
cv_acc=cv_acc.mean()
acc=accuracy_score(y_test,ypred)
acc1=f1_score(y_test,ypred,average='binary')
auc = roc_auc_score(y_test,ypred)
aps=average_precision_score(y_test,ypred)

In [45]:
print('Confusion Matrix\n',cm)
print('\nCross Validation Score\n',cv_acc)
print('\nTesting Accuracy\n',acc)
print('\nF1 Score\n',acc1)
print('\nAUC\n',auc)
print('\nAverage Precision Score\n',aps)

Confusion Matrix
 [[3373   42]
 [  84 1292]]

Cross Validation Score
 0.9732814143075588

Testing Accuracy
 0.9737006887914841

F1 Score
 0.9535055350553505

AUC
 0.9633274030440262

Average Precision Score
 0.9269241087542611


__Model 2: Randomforest Classifier__

In [37]:
from sklearn.ensemble import RandomForestClassifier

model1=RandomForestClassifier(random_state=0,n_jobs=-1,n_estimators=90)
model1.fit(x_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=90, n_jobs=-1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [0]:
# Predicting Output

ypred1=model1.predict(x_test)

In [0]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score,confusion_matrix,f1_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import average_precision_score

cv_acc=cross_val_score(estimator=model1,X=x_train,y=y_train,cv=5)
cm=confusion_matrix(y_test,ypred1)
cv_acc=cv_acc.mean()
acc=accuracy_score(y_test,ypred1)
acc1=f1_score(y_test,ypred1,average='binary')
auc = roc_auc_score(y_test,ypred1)
aps=average_precision_score(y_test,ypred1)

In [40]:
print('Confusion Matrix\n',cm)
print('\nCross Validation Score\n',cv_acc)
print('\nTesting Accuracy\n',acc)
print('\nF1 Score\n',acc1)
print('\nAUC\n',auc)
print('\nAverage Precision Score\n',aps)

Confusion Matrix
 [[3396   19]
 [  33 1343]]

Cross Validation Score
 0.988310746868714

Testing Accuracy
 0.9891463160091839

F1 Score
 0.9810080350620891

AUC
 0.98522687612789

Average Precision Score
 0.9692898417262321


__ Hence from the analysis RandomForest Classifier provides better result.Therefore we will use randomforest classifier above logistic regression.__
